In [52]:
import pandas as pd
import logging


logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="Sales_by_product_log.log",filemode="w")

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv',parse_dates=['payment_date'])
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv',parse_dates=['created_at'])
#venue=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\cafe.csv',parse_dates=['last_update'])
#address=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\address.csv',parse_dates=['last_update'])
product=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\item.csv',parse_dates=['last_update'])

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=False)
#merged_df=merged_df.merge(venue,on='cafe_id',how='left',indicator=False)
#merged_df=merged_df.merge(address,on='address_id',how='left',indicator=False)
merged_df=merged_df.merge(product,on='item_id',how='left',indicator=False)



merged_df=merged_df.drop(columns=['payment_id','customer_id_x','emp_id','payment_type',
                        'order_id','item_id','customer_id_y','delivery',
                        'created_at','customer_modifications','delivery_notes',
                        'recipe_id',
                        'amount','item_size'])

merged_df.head()

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte
merged_df['Year']=payment_date_converted.year
merged_df['Year']=merged_df['Year'].astype('str')

merged_df['Month']=payment_date_converted.month 
merged_df['Month']=merged_df['Month'].astype('str')
merged_df['Month'] = merged_df['Month'].str.zfill(2)

merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

merged_df=merged_df.drop(columns=['payment_date'])
#venue_dictionary={1:"New York",2:"Miami",3:"Lima",4:"Buenos Aires",5:"Seattle"}
#merged_df['cafe_id']=merged_df['cafe_id'].map(venue_dictionary)
#region_dictionary={"Peru":"LATAM","Argentina":"LATAM","USA":"North America"}
#merged_df['region']=merged_df['country'].map(region_dictionary)
product_category_dictionary={"butter croissant":"food","cinnamon bun":"food", "multigrain toast with butter":"food",
                             "iced coffee":"drinks","hot coffee":"drinks"}
merged_df['product_category']=merged_df['item_name'].map(product_category_dictionary)
merged_df.rename(columns={'item_name':'product_name','item_price':'product_price',
                          'item_quantity':'qty_sold',
                          'Year':'year','Year_Month':'year_month','Month':'month' },inplace=True)


merged_df=merged_df.loc[:,['year','year_month','month',
                           'product_category','product_name',
                           'product_price','qty_sold']]


#merged_df['product_size']=merged_df['product_size'].fillna("one size")

merged_df['sales']=merged_df['product_price']*merged_df['qty_sold']

monthly_product_sales_df=merged_df.groupby(['product_name','year_month','year','month']).agg(
    monthly_sales=('sales','sum'),
    monthly_qty_sold=('qty_sold','sum'),
).reset_index()
###
#monthly_product_sales_df=monthly_product_sales_df.drop(columns=['product_price','monthly_qty_sold'])
####
avg_monthly_product_sales_df=monthly_product_sales_df.groupby(['product_name','month']).agg(
    avg_product_sales_by_month_of_the_year=('monthly_sales','mean')
).reset_index()


monthly_product_sales_df=pd.merge(merged_df,monthly_product_sales_df,how='right',on=['product_name',
                                                                                     'year_month','year','month'])
monthly_product_sales_df=monthly_product_sales_df.drop(columns=['sales','qty_sold','monthly_qty_sold','product_price'])  ###################


monthly_product_sales_df=pd.merge(monthly_product_sales_df,avg_monthly_product_sales_df,how='left',on=['product_name','month'] )
monthly_product_sales_df['avg_product_sales_by_month_of_the_year']=monthly_product_sales_df['avg_product_sales_by_month_of_the_year'].apply(lambda x:round(x,2))


monthly_product_sales_df=monthly_product_sales_df.loc[:,['product_name','product_category','year',
                                'year_month','month',
                               'monthly_sales','avg_product_sales_by_month_of_the_year']]


monthly_product_sales_df['monthly_sales']=monthly_product_sales_df['monthly_sales'].apply(lambda x:round(x,2))
monthly_product_sales_df=monthly_product_sales_df.drop_duplicates()

monthly_total_sales_df=monthly_product_sales_df.groupby(['year_month','year','month']).agg(
    total_monthly_sales=('monthly_sales','sum')  
)

sales_by_category_df=monthly_product_sales_df.groupby(['product_category','year_month','year','month']).agg(
    monthly_sales_of_this_product_category=('monthly_sales','sum')
    
)



sales_by_category_df=sales_by_category_df.reset_index()

monthly_product_sales_df=pd.merge(monthly_product_sales_df,sales_by_category_df,how='left', on=['product_category','year_month','year','month'])
total_monthly_sales_df=monthly_product_sales_df.groupby(['year_month','year','month']).agg(
    total_monthly_sales=('monthly_sales','sum')    
)
sales_by_product_df=monthly_product_sales_df.groupby(['product_name','year_month','year','month']).agg(
    product_monthly_sales=('monthly_sales','sum')  
)




sales_by_product_df=sales_by_product_df.reset_index()
monthly_product_sales_df=pd.merge(monthly_product_sales_df,sales_by_product_df,how='left', on=['product_name','year_month','year','month'])
total_monthly_sales_df=total_monthly_sales_df.reset_index()
monthly_product_sales_df=pd.merge(monthly_product_sales_df,total_monthly_sales_df,how='left',on=['year_month','year','month'])

monthly_product_sales_df['percentage_category_sales_from_total_sales']=monthly_product_sales_df['monthly_sales_of_this_product_category']/monthly_product_sales_df['total_monthly_sales']*100
monthly_product_sales_df['percentage_category_sales_from_total_sales']=monthly_product_sales_df['percentage_category_sales_from_total_sales'].apply(lambda x: round(x,2))

annual_sales_by_category_df=merged_df.groupby(['product_category','year']).agg(
    annual_sales_by_category=('sales','sum')
)
annual_sales_by_category_df=annual_sales_by_category_df.reset_index()
annual_sales_total_df=pd.DataFrame(annual_sales_by_category_df.groupby(['year'])['annual_sales_by_category'].sum())
annual_sales_total_df=annual_sales_total_df.reset_index()
annual_sales_total_df=annual_sales_total_df.rename(columns={'annual_sales_by_category':'annual_sales_total'})

annual_sales_by_category_df=pd.merge(annual_sales_by_category_df,annual_sales_total_df,how='left',on='year')
annual_sales_by_category_df['category_sales_perc_from_total']=annual_sales_by_category_df['annual_sales_by_category']/annual_sales_by_category_df['annual_sales_total']
annual_sales_by_category_df['category_sales_perc_from_total']=annual_sales_by_category_df['category_sales_perc_from_total'].apply(lambda x: round(x,2))

annual_sales_by_product_df=merged_df.groupby(['product_name','year']).agg(
    annual_sales_by_product=('sales','sum')
)
annual_sales_by_product_df=annual_sales_by_product_df.reset_index()
annual_sales_total_df=pd.DataFrame(annual_sales_by_product_df.groupby(['year'])['annual_sales_by_product'].sum())
annual_sales_total_df=annual_sales_total_df.reset_index()
annual_sales_total_df=annual_sales_total_df.rename(columns={'annual_sales_by_product':'annual_sales_total'})
annual_sales_total_df.head()
annual_sales_by_product_df=pd.merge(annual_sales_by_product_df,annual_sales_total_df,how='left',on='year')
annual_sales_by_product_df['product_sales_perc_from_total']=annual_sales_by_product_df['annual_sales_by_product']/annual_sales_by_product_df['annual_sales_total']*100
annual_sales_by_product_df['product_sales_perc_from_total']=annual_sales_by_product_df['product_sales_perc_from_total'].apply(lambda x: round(x,2))


monthly_product_sales_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Monthly_sales_by_product.csv',index=False)
monthly_product_sales_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\Monthly_sales_by_product.csv',index=False)
#sales_by_category_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\sales_by_category.csv',index=False)
annual_sales_by_product_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Annual_sales_by_product.csv',index=False)
annual_sales_by_product_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\Annual_sales_by_product.csv',index=False)
annual_sales_by_category_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Annual_sales_by_product_category.csv',index=False)
annual_sales_by_category_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\Annual_sales_by_product_category.csv',index=False)


In [53]:
monthly_product_sales_df[monthly_product_sales_df['product_name']=="hot coffee"]

,product_name,product_category,year,year_month,month,monthly_sales,avg_product_sales_by_month_of_the_year,monthly_sales_of_this_product_category,product_monthly_sales,total_monthly_sales,percentage_category_sales_from_total_sales
122,hot coffee,drinks,2020,2020-03,03,2230.42,1997.66,4633.77,2230.42,6207.74,74.65
123,hot coffee,drinks,2020,2020-04,04,2272.35,2506.68,5059.31,2272.35,6624.32,76.37
124,hot coffee,drinks,2020,2020-05,05,2661.89,2487.09,5348.97,2661.89,7112.76,75.20
125,hot coffee,drinks,2020,2020-06,06,2789.73,2540.83,5231.01,2789.73,7113.61,73.54
126,hot coffee,drinks,2020,2020-07,07,1948.71,2464.52,4116.32,1948.71,5572.47,73.87
...,...,...,...,...,...,...,...,...,...,...,...
178,hot coffee,drinks,2024,2024-11,11,3067.40,2532.04,6110.08,3067.40,7963.71,76.72
179,hot coffee,drinks,2024,2024-12,12,2249.37,2268.14,4203.24,2249.37,5835.22,72.03
180,hot coffee,drinks,2025,2025-01,01,2779.73,2460.13,5456.80,2779.73,7184.69,75.95
181,hot coffee,drinks,2025,2025-02,02,2305.29,2172.05,4261.15,2305.29,6069.79,70.20
